# Prereqs

First thing that we're going to do is to set up and install of the prereqs.


TODO: Determine which aren't needed

In [1]:
!pip install --upgrade pip opencv-python pyqt5 lxml wget

     |████████████████████████████████| 1.6 MB 5.1 MB/s 
     |████████████████████████████████| 49.9 MB 15 kB/s 
     |████████████████████████████████| 8.3 MB 44.2 MB/s 
     |████████████████████████████████| 6.3 MB 40.2 MB/s 
     |████████████████████████████████| 59.9 MB 40 kB/s 
     |████████████████████████████████| 317 kB 49.1 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=6fc1bc3797cc401a0bcc546c2527df7cf2882a11e2e21f2e0d2ba1c80cd2d8a3
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
  Attempting uninstall: lxml
    Found existing i

These are the global variables that we'll need

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

BASE_DIRECTORY=os.path.join(os.getcwd(),'drive','MyDrive','TensorFlow')

CUSTOM_MODEL_NAME = 'my_centernet' 
PRETRAINED_MODEL_NAME = 'centernet_hg104_512x512_kpts_coco17_tpu-32'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'


paths = {
    'WORKSPACE_PATH': os.path.join(BASE_DIRECTORY, 'workspace'),
    'SCRIPTS_PATH': os.path.join(BASE_DIRECTORY,'scripts'),
    'APIMODEL_PATH': os.path.join(BASE_DIRECTORY,'models'),
    'ANNOTATION_PATH': os.path.join(BASE_DIRECTORY, 'workspace','annotations'),
    'IMAGE_PATH': os.path.join(BASE_DIRECTORY, 'workspace','images'),
    'PRETRAINED_MODEL_PATH': os.path.join(BASE_DIRECTORY, 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join(BASE_DIRECTORY, 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join(BASE_DIRECTORY, 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join(BASE_DIRECTORY, 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join(BASE_DIRECTORY, 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join(BASE_DIRECTORY,'protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join(paths['WORKSPACE_PATH'],'models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

TRAIN_IMAGES = os.path.join(paths['IMAGE_PATH'], 'MO17D_419', 'Half_Size_1632x1224')
TEST_IMAGES = os.path.join(paths['IMAGE_PATH'], 'MO17D_611', 'Half_Size_1632x1224')

TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

Mounted at /content/drive


Make sure all directories exist

In [3]:
for path in paths.values():
  os.makedirs(path, exist_ok=True)

Change directories

In [4]:
os.chdir(BASE_DIRECTORY)

Check if we need to clone repos. If we do, then do it.

In [5]:
if not os.path.isdir(os.path.join(BASE_DIRECTORY, 'models')):
  !git clone https://github.com/tensorflow/models.git
#Update the paths array
paths['MODEL_PATH'] = os.path.join(BASE_DIRECTORY, 'workspace','models')

In [6]:
!pwd

/content/drive/MyDrive/TensorFlow


# TensorFlow API installation

In [7]:
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd {BASE_DIRECTORY}/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd {BASE_DIRECTORY}/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd {BASE_DIRECTORY}/models/research/slim && pip install -e .

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Processing /content/drive/MyDrive/TensorFlow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 8.6 MB/s 
     |████████████████████████████████| 352 kB 46.4 MB/s 
     |████████████████████████████████| 1.8 MB 39.5 MB/s 
     |████████████████████████████████| 211 kB 43.7 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |██████████████████████

Verify

In [8]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
!python {VERIFICATION_SCRIPT}

2021-09-14 20:06:27.698612: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-14 20:06:28.283853: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-14 20:06:28.284691: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.7.11: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-09-14 20:06:28.299927: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021

Upgrade Tensorflow

In [9]:
%pip install tensorflow --upgrade
%pip uninstall protobuf matplotlib -y
%pip install protobuf matplotlib==3.2

Found existing installation: protobuf 3.17.3
Uninstalling protobuf-3.17.3:
  Successfully uninstalled protobuf-3.17.3
Found existing installation: matplotlib 3.2.2
Uninstalling matplotlib-3.2.2:
  Successfully uninstalled matplotlib-3.2.2
     |████████████████████████████████| 1.0 MB 5.3 MB/s 
     |████████████████████████████████| 12.4 MB 168 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


#Training

Grab a model

TODO: check if the downloaded file exists. Downloading a new model takes seconds, so it's not a top priority

In [ ]:
import tarfile, wget
downloaded_file = wget.download(PRETRAINED_MODEL_URL, out=paths['PRETRAINED_MODEL_PATH'])
tar = tarfile.open(downloaded_file)
tar.extractall(path=paths['PRETRAINED_MODEL_PATH'])
tar.close()

  Using cached wget-3.2.zip (10 kB)
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=e639e14f07575c826d4c4b9857d127d98270f338dae380d85a0ba5079929533e
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


Create the label map

In [ ]:
labels = [{'name':'Oligodendrocyte', 'id':1}, {'name':'Question', 'id':2}, {'name':'Interneuron', 'id':3}, {'name':'Pyramidal', 'id':4}, {'name':'Axon', 'id':5}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

Get the tfrecord script

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

!python {files['TF_RECORD_SCRIPT']} -i {TRAIN_IMAGES} -x {os.path.join(TRAIN_IMAGES, 'XML')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -i {TEST_IMAGES} -x {os.path.join(TEST_IMAGES, 'XML')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}

Copy and setup the pipeline config
#TODO this section is broken.

In [ ]:
import shutil
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format



shutil.copy(os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config'), os.path.join(paths['CHECKPOINT_PATH']))

config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

config

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]



config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])
!{command}

2021-09-14 20:07:28.741641: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-14 20:07:28.757742: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-14 20:07:28.758549: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-14 20:07:28.760040: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-14 20:07:28.760767: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [ ]:
print(BASE_DIRECTORY)

/content/drive/MyDrive/TensorFlow


#Evaluation

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])